## Find the number of gates and calculate the quantum cost of the circuit.

In [1]:
from qiskit import *

In [2]:
#string to be found is 01011001
circuit=QuantumCircuit(16,8)
a=[0,1,2,3,4,5,6,7,15]
b=[0,1,2,3,4,5,6,7]
circuit.x(15)
circuit.h(a)
i=0
while i<12:
    i=i+1
    #oracle
    circuit.x([1,2,5,7])
    circuit.ccx(0,1,8)
    x=2
    y=8
    z=9
    while x<8:
        circuit.ccx(x,y,z)
        x=x+1
        y=y+1
        z=z+1
    circuit.cx(14,15)
    while x>2:
        x=x-1
        y=y-1
        z=z-1
        circuit.ccx(x,y,z)
    circuit.ccx(0,1,8)
    circuit.x([1,2,5,7])

    #grover diffusion operator
    circuit.h(b)
    circuit.x(b)
    circuit.ccx(0,1,8)
    x=2
    y=8
    z=9
    while x<8:
        circuit.ccx(x,y,z)
        x=x+1
        y=y+1
        z=z+1
    circuit.cx(14,15)
    while x>2:
        x=x-1
        y=y-1
        z=z-1
        circuit.ccx(x,y,z)
    circuit.ccx(0,1,8)  
    circuit.x(b)
    circuit.h(b)

circuit.measure(b,b)

### -Number of gate:
**Use the `count_ops()` method to harvest the number of operations in the circuit.**

In [3]:
########### Count the operators ###########
circuit.count_ops()

OrderedDict([('ccx', 336), ('x', 289), ('h', 201), ('cx', 24), ('measure', 8)])

### -Quantum Cost:
The quantum cost of a circuit is the number of primitive quantum gates needed to implement the circuit.

And here we go!

There are: 

* $336$ Toffoli Gate `ccx`.
* $289$ Not Gate `x`.
* $201$ Hadamard Gate `h`.
* $24$ Control Gate `cx`.

**The costs of Gates `x`, `h` and `cx` are said to be $1$, and that of `ccx` is $5$.**

Each quantum computer model may have a different cost for a given quantum gate.

$\rightarrow$ hence the quantum cost of that circuit is:$$Quantum\;Cost = 5\times336 + 1\times289 + 1\times201 + 1\times24 = 2194.$$

Actually, let us decompose the circuit to the primitive gates (elementary gate), like the rotation gates U3 and the CNOT gate, then calculate the cost.

Any unitary operation can be replaced by a combination of controlled-NOT (CNOT) and single qubit operations, or gates U1,2,3

By using `qiskit.transpiler` and importing `PassManager`, we can decompose this circuit into U3 (`u3`) gates and CNOT (`cx`) gates using the qiskit tool `Unroller` program, as seen below:

In [4]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(circuit) 
new_circuit.count_ops()

OrderedDict([('u3', 3514), ('cx', 2040), ('measure', 8)])

There are: 

* $336$ U3 Gate `u3`.
* $2040$ Ctrolled Gate `cx`.

$\rightarrow$ Terefore, the quantum cost of that circuit in thos case is:$$Quantum\;Cost = 336 + 2040= 2376$$

Whereas, the CNOT gate is the most expensive gate to implement, wherein the current NISQ devises the noise introduced implementing the CX gate is heavier.

Therefore, we may weigh CX gates more than a single-qubit gate for cost evaluation.

**The lesser the quantum cost, the better the circuit.**